In [2]:
import time
import numpy as np
import math as math
from functools import reduce
from random import randint
from random import uniform
from math import pi,sqrt,cos,sin

In [3]:
i = 0
N = 10 #Tamanho da vizinhanca
NumeroItens = 500
MaxWeight = 10.0
MinWeight = 1.0

#weights = np.arange(MinWeight,MaxWeight)
itens = np.random.uniform(low=MinWeight,high = MaxWeight, size=(NumeroItens,))

#itens = np.zeros(NumeroItens)
#for i in range(NumeroItens):
#    itens[i] = weights[randint(0,MaxWeight - MinWeight - 1)]

C = reduce(lambda x,y : x+y, itens)/2
profits = np.vectorize(lambda x: x + 5)(itens)

print("Espaço de itens: ",itens)
print("Ganho dos itens", profits)
print("Capacidade da mochila: ", C)

Espaço de itens:  [ 6.09119158  9.07276748  6.48573143  8.26557831  2.06876933  7.30119429
  4.06568238  4.8362218   8.46926848  5.44699012  5.45907848  5.82288365
  7.77083493  4.92149531  8.99281686  3.90082661  5.96612194  9.46161566
  2.251199    1.54241984  5.48996508  5.30731775  2.32921851  8.9943199
  2.51061576  9.12181774  6.18749384  6.40743328  2.17512473  3.00608313
  6.18490245  2.72191113  9.11917725  6.81996059  2.80020735  3.65452222
  4.71582223  4.2558488   9.80797393  1.9579592   5.94561224  6.66739838
  8.74856668  9.45958115  1.10270018  7.49672459  7.60647972  8.99759909
  3.19319571  7.56583422  4.51938457  2.794005    1.62195332  3.94686494
  3.36076524  4.37354033  4.46755157  9.27027129  4.45656566  6.5600566
  5.59290705  1.51909181  7.43158599  3.82504233  7.72947788  4.42836373
  4.78929389  7.19510695  7.48765289  4.66368789  5.39458217  6.88939139
  6.12200057  9.67413167  4.31428498  8.82171693  8.25728535  1.71079898
  8.09133765  6.42856705  5.3563294

In [ ]:
def ObjFun(profits, solution):
    return reduce(lambda x,y: x+y, 
                  np.vectorize(lambda x,y: x*y)(profits, solution))

def GerarVizinhos(qindividuos, N):
    Vizinhos = np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                                    (qindividuos[:,1], np.random.rand(NumeroItens))
    for i in range(N-1):
        NewN = np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                                    (qindividuos[:,1], np.random.rand(NumeroItens))
        Vizinhos = np.vstack([Vizinhos, NewN])
    return Vizinhos

#Greedy Repair, Lamarckian
def Ajustar(vizinhos,C):
    for i in range(vizinhos.shape[0]):
        itensSelected = vizinhos[i,:].nonzero()[0]
        while (ObjFun(profits, vizinhos[i,:]) > C):
            r = randint(0,itensSelected.shape[0]-1)
            j = itensSelected[r]
            vizinhos[i,j] = 0
            itensSelected = np.delete(itensSelected, r)
    return vizinhos

def AjustarSolucao(solucao,C):
    itensSelected = solucao.nonzero()[0]
    while (ObjFun(profits, solucao) > C):
        r = randint(0,itensSelected.shape[0]-1)
        j = itensSelected[r]
        solucao[j] = 0
        itensSelected = np.delete(itensSelected, r)
    return solucao

def NewBestFit(vizinhos, bestFit):
    melhorPontuacao = ObjFun(profits, vizinhos[0,:])
    indexMelhorSol = 0
    for i in range(1,vizinhos.shape[0]):
        novaPontuacao = ObjFun(profits, vizinhos[i,:])
        if (novaPontuacao > melhorPontuacao):
            melhorPontuacao = novaPontuacao
            indexMelhorSol = i
    if (melhorPontuacao > ObjFun(profits,bestFit)):
        bestFit = vizinhos[indexMelhorSol,:]
    return bestFit

def FindBest(vizinhos):
    melhorValor = ObjFun(profits, vizinhos[0,:])
    melhorSol = 0
    for i in range(1,vizinhos.shape[0]):
        novoValor = ObjFun(profits,vizinhos[i,:])
        if (novoValor > melhorValor):
            melhorValor = novoValor
            melhorSol = i
    return vizinhos[melhorSol,:]

def FindWorst(vizinhos):
    piorValor = ObjFun(profits, vizinhos[0,:])
    piorSol = 0
    for i in range(1,vizinhos.shape[0]):
        novoValor = ObjFun(profits, vizinhos[i,:])
        if (novoValor > piorValor):
            piorValor = novoValor
            piorSol = i
    return vizinhos[piorSol,:]

#Retorna o que NAO É TABU
def AtualizarTabu(piorSol, melhorSol):
    return [i for i in range(piorSol.size) if(melhorSol[i] ^ piorSol[i])] #xor

def AtualizarQ(sol,piorSol, melhorSol, qindividuos):
    theta = 0.01*pi
    
    nonTabuQbits = AtualizarTabu(sol, melhorSol)
    cplus = cos(theta)
    splus = sin(theta)
    
    cminus = cos(-theta)
    sminus = sin(-theta)
    for i in nonTabuQbits:
        if (qindividuos[i,0]*qindividuos[i,1] > 0):
            #Qbit[i] está no primeiro ou terceiro quadrante
            if (sol[i] == 1):
                c = cplus
                s = splus
            else:
                c = cminus
                s = sminus
        else:
            #Qbit[i] está no segundo ou quarto quadrante
            if (sol[i] == 1):
                c = cminus
                s = sminus
            else:
                c = cplus
                s = splus
            
        Ugate = np.array([[c, -s],
                          [s,  c]])  #Matriz de rotação    
        #Rotação do individuo i
        qindividuos[i,:] = np.dot(Ugate,qindividuos[i,:])

    nonTabuQbits = AtualizarTabu(sol, piorSol)
    sol = np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                      (qindividuos[:,0], np.random.rand(NumeroItens))
    
    for i in nonTabuQbits:
        if (qindividuos[i,0]*qindividuos[i,1] > 0):
            #Qbit[i] está no primeiro ou terceiro quadrante
            if (sol[i] == 1):
                c = cminus
                s = sminus
            else:
                c = cplus
                s = splus
        else:
            #Qbit[i] está no segundo ou quarto quadrante
            if (sol[i] == 1):
                c = cplus
                s = splus
            else:
                c = cminus
                s = sminus
            
        Ugate = np.array([[c, -s],
                          [s,  c]])  #Matriz de rotação    
        #Rotação do individuo i
        qindividuos[i,:] = np.dot(Ugate,qindividuos[i,:])
    return qindividuos


In [ ]:
start_time = time.time()
qindividuos = np.zeros((NumeroItens,2))
qindividuos.fill(1/sqrt(2))
solucao = np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                      (qindividuos[:,0], np.random.rand(NumeroItens))
bestFit = solucao

i = 0
NumIter = 1000
print("Limite da mochila: ", C)
print("Nro de iteracoes: ", NumIter)
print("Numero de itens: ", NumeroItens)
print("Ganho Inicial:(Sem Reparo)", ObjFun(profits,bestFit))
bestFit = AjustarSolucao(bestFit,C)
print("Ganho Iniciral:(Com Reparo)", ObjFun(profits,bestFit))
while (i < NumIter):
    i = i + 1 
    vizinhos = GerarVizinhos(qindividuos, N) 
    vizinhos = Ajustar(vizinhos, C)
    
    bestFit = NewBestFit(vizinhos,bestFit)
    melhorSolucao = FindBest(vizinhos)
    piorSolucao = FindWorst(vizinhos)
    qindividuos = AtualizarQ(solucao, melhorSolucao, piorSolucao, qindividuos)
    
    solucao = np.vectorize(lambda x,y : 1 if (x > np.power(y,2)) else 0)\
                      (qindividuos[:,0], np.random.rand(NumeroItens))

print("Ganho Melhor Solucao Encontrada: ", ObjFun(profits,bestFit))
print("Tempo : ", (time.time() - start_time))

Limite da mochila:  1357.09953431
Nro de iteracoes:  1000
Numero de itens:  500
Ganho Inicial:(Sem Reparo) 4323.62418298
Ganho Iniciral:(Com Reparo) 1346.28773037
